In [64]:
import pandas as pd
import numpy as np
import os

In [65]:
explore_turn = pd.read_csv("turn_preprocessed.csv")
explore_user = pd.read_csv("user_preprocessed.csv")

#### 2149 conversation turns

In [66]:
explore_turn['utterance_id'].value_counts()

utterance_id
1       1
1446    1
1460    1
1459    1
1458    1
       ..
712     1
711     1
710     1
709     1
2186    1
Name: count, Length: 2149, dtype: int64

#### 185 chats

In [67]:
explore_turn['chat_id'].value_counts()

chat_id
169.0    15
171.0    15
1.0      12
124.0    12
116.0    12
         ..
165.0     5
77.0      5
187.0     5
29.0      2
184.0     2
Name: count, Length: 185, dtype: int64

#### 164 participants

In [68]:
submission_id_count = explore_user['Submission id'].value_counts()
duplicate_submission_ids = submission_id_count[submission_id_count > 1].index.to_list()
print(submission_id_count)
print(duplicate_submission_ids)

Submission id
64dcc1d6ef8aa02ac3c54749    4
64d3350ab09727a9eca1450b    3
64da2e7a58e166cb776329ff    2
64d107ebc5b615bf51ff3c31    2
64dc97f0509832556ec5477c    2
                           ..
64d33dbe6f422d203f9acfde    1
64dcbe573059a70a9ee178a5    1
64d36441d2b0449574ac9ed9    1
64d3468509eecf544c8584e8    1
64dc7992fd5c8149d3b5d4f2    1
Name: count, Length: 164, dtype: int64
['64dcc1d6ef8aa02ac3c54749', '64d3350ab09727a9eca1450b', '64da2e7a58e166cb776329ff', '64d107ebc5b615bf51ff3c31', '64dc97f0509832556ec5477c', '64d1f6c67356bab250759725', '64da1e3a65f69daf234e62f5', '64d21a7c747a2537c8e97211', '64d202f7b79c610681113153', '64d22263f27eb3d7005fe341', '64da2757c44ff417ef2fab96', '64dc992a8728d05e89c86d12', '64da2de18f089e007ce53829']


#### 13 users submitted chats multipal times, we have 16 muitipal chats

In [69]:
explore_user['Submission id'].value_counts().head(15)

Submission id
64dcc1d6ef8aa02ac3c54749    4
64d3350ab09727a9eca1450b    3
64da2e7a58e166cb776329ff    2
64d107ebc5b615bf51ff3c31    2
64dc97f0509832556ec5477c    2
64d1f6c67356bab250759725    2
64da1e3a65f69daf234e62f5    2
64d21a7c747a2537c8e97211    2
64d202f7b79c610681113153    2
64d22263f27eb3d7005fe341    2
64da2757c44ff417ef2fab96    2
64dc992a8728d05e89c86d12    2
64da2de18f089e007ce53829    2
64d48ce79ecbb8009fb2afa9    1
64d5dd779bec7ce53e607cf8    1
Name: count, dtype: int64

#### Issue: 
in **turn_preprocessed.csv** we had 185 unique chat_id, but in **user_preprocessed.csv**, we have 176 unique chat_id, which means the two files doesn't match

In [70]:
explore_user['chat_id'].value_counts()

chat_id
164    2
173    2
159    2
174    2
33     1
      ..
88     1
188    1
43     1
47     1
168    1
Name: count, Length: 176, dtype: int64

#### **check problematic chat_id in user_preprocessed.csv**

In [71]:
chat_id_counts = explore_user['chat_id'].value_counts()
# Filter out `chat_id` values that appear more than once and convert them to a list.
duplicate_chat_ids = chat_id_counts[chat_id_counts > 1].index.to_list()
print(duplicate_chat_ids)

[164, 173, 159, 174]


Different Submission id, same user, same chat_id

In [72]:
explore_user[explore_user['chat_id'] == 164]

,Unnamed: 0,Submission id,condition,Age,Sex,education,job,start_time,end_time,Time taken,...,Perception of MI,User Engagement - Focused Attention,User Engagement - Perceived Usability,User Engagement - Reward,User Engagement - Overall,Communication Competence,Perceived Empathy,Readiness to Change (Delta),Stage of Change (Start),chat_id
71,71,64dc97f0509832556ec5477c,no_system,28,Male,Abitur,Ergotherapeut,2023-08-14 16:19:42.48471,2023-08-16 11:48:22.540476,14.800000,...,29.0,3.666667,5.0,3.333333,4.0,4.0,1.0,0.0,contemplation,164
73,73,64da2e7a58e166cb776329ff,no_system,28,Male,Abitur,Ergotherapeut,2023-08-14 16:19:42.48471,2023-08-16 11:48:22.540476,55.433333,...,29.0,3.666667,5.0,3.333333,4.0,4.0,1.0,0.0,contemplation,164


Same submission id, same user, same conversation, different chat_id

In [73]:
explore_user[explore_user['chat_id'] == 173]

,Unnamed: 0,Submission id,condition,Age,Sex,education,job,start_time,end_time,Time taken,...,Perception of MI,User Engagement - Focused Attention,User Engagement - Perceived Usability,User Engagement - Reward,User Engagement - Overall,Communication Competence,Perceived Empathy,Readiness to Change (Delta),Stage of Change (Start),chat_id
72,72,64dc97f0509832556ec5477c,no_system,28,Male,Abitur,Ergotherapeut,2023-08-14 16:19:42.48471,2023-08-16 11:48:22.540476,14.800000,...,29.0,3.666667,5.0,3.333333,4.0,4.0,1.0,0.0,contemplation,173
74,74,64da2e7a58e166cb776329ff,no_system,28,Male,Abitur,Ergotherapeut,2023-08-14 16:19:42.48471,2023-08-16 11:48:22.540476,55.433333,...,29.0,3.666667,5.0,3.333333,4.0,4.0,1.0,0.0,contemplation,173


In [74]:
explore_user[explore_user['chat_id'] == 159]

,Unnamed: 0,Submission id,condition,Age,Sex,education,job,start_time,end_time,Time taken,...,Perception of MI,User Engagement - Focused Attention,User Engagement - Perceived Usability,User Engagement - Reward,User Engagement - Overall,Communication Competence,Perceived Empathy,Readiness to Change (Delta),Stage of Change (Start),chat_id
6,6,64dc992a8728d05e89c86d12,framework,28,Male,Abitur,selbstständig,2023-08-14 15:22:59.472128,2023-08-16 12:04:35.726276,25.216667,...,37.0,2.666667,4.333333,3.666667,3.555556,4.0,3.666667,0.0,action,159
8,8,64da2757c44ff417ef2fab96,framework,28,Male,Abitur,selbstständig,2023-08-14 15:22:59.472128,2023-08-16 12:04:35.726276,39.683333,...,37.0,2.666667,4.333333,3.666667,3.555556,4.0,3.666667,0.0,action,159


In [75]:
explore_user[explore_user['chat_id'] == 174]

,Unnamed: 0,Submission id,condition,Age,Sex,education,job,start_time,end_time,Time taken,...,Perception of MI,User Engagement - Focused Attention,User Engagement - Perceived Usability,User Engagement - Reward,User Engagement - Overall,Communication Competence,Perceived Empathy,Readiness to Change (Delta),Stage of Change (Start),chat_id
7,7,64dc992a8728d05e89c86d12,framework,28,Male,Abitur,selbstständig,2023-08-14 15:22:59.472128,2023-08-16 12:04:35.726276,25.216667,...,37.0,2.666667,4.333333,3.666667,3.555556,4.0,3.666667,0.0,action,174
9,9,64da2757c44ff417ef2fab96,framework,28,Male,Abitur,selbstständig,2023-08-14 15:22:59.472128,2023-08-16 12:04:35.726276,39.683333,...,37.0,2.666667,4.333333,3.666667,3.555556,4.0,3.666667,0.0,action,174


#### **Solution**
**Delete all those rows. from 176 chat_id to 172 chat_id**

In [76]:
explore_user_cleaned = explore_user[~explore_user['chat_id'].isin(duplicate_chat_ids)]
chat_id_counts = explore_user_cleaned['chat_id'].value_counts()
chat_id_counts

chat_id
62     1
34     1
30     1
45     1
36     1
      ..
88     1
188    1
43     1
47     1
168    1
Name: count, Length: 172, dtype: int64

**from 164 Submission id to 160  Submission id**

In [77]:
Submission_id_counts = explore_user_cleaned['Submission id'].value_counts()
Submission_id_counts

Submission id
64dcc1d6ef8aa02ac3c54749    4
64d3350ab09727a9eca1450b    3
64d202f7b79c610681113153    2
64d22263f27eb3d7005fe341    2
64da2de18f089e007ce53829    2
                           ..
64d33dbe6f422d203f9acfde    1
64dcbe573059a70a9ee178a5    1
64d36441d2b0449574ac9ed9    1
64d3468509eecf544c8584e8    1
64dc7992fd5c8149d3b5d4f2    1
Name: count, Length: 160, dtype: int64

#### **Check multiple submission from same Submission id in user_preprocessed.csv**

In [78]:
explore_user_cleaned[explore_user_cleaned['Submission id'] == '64dcc1d6ef8aa02ac3c54749']

,Unnamed: 0,Submission id,condition,Age,Sex,education,job,start_time,end_time,Time taken,...,Perception of MI,User Engagement - Focused Attention,User Engagement - Perceived Usability,User Engagement - Reward,User Engagement - Overall,Communication Competence,Perceived Empathy,Readiness to Change (Delta),Stage of Change (Start),chat_id
75,75,64dcc1d6ef8aa02ac3c54749,framework,27,Male,Bachelor-Abschluss,Student,2023-08-14 16:19:21.992909,2023-08-16 15:18:54.435796,46.416667,...,33.0,2.333333,3.666667,3.666667,3.222222,3.25,3.666667,0.0,contemplation,165
76,76,64dcc1d6ef8aa02ac3c54749,framework,27,Male,Bachelor-Abschluss,Student,2023-08-14 16:19:21.992909,2023-08-16 15:18:54.435796,46.416667,...,33.0,2.333333,3.666667,3.666667,3.222222,3.25,3.666667,0.0,contemplation,166
77,77,64dcc1d6ef8aa02ac3c54749,framework,27,Male,Bachelor-Abschluss,Student,2023-08-14 16:19:21.992909,2023-08-16 15:18:54.435796,46.416667,...,33.0,2.333333,3.666667,3.666667,3.222222,3.25,3.666667,0.0,contemplation,184
78,78,64dcc1d6ef8aa02ac3c54749,framework,27,Male,Bachelor-Abschluss,Student,2023-08-14 16:19:21.992909,2023-08-16 15:18:54.435796,46.416667,...,33.0,2.333333,3.666667,3.666667,3.222222,3.25,3.666667,0.0,contemplation,185


In [79]:
chat_ids = explore_user_cleaned[explore_user_cleaned['Submission id'] == '64dcc1d6ef8aa02ac3c54749']['chat_id'].to_list()
chat_ids

[165, 166, 184, 185]

#### Look at those chat_id in turn_preprocess.csv

In [80]:
explore_turn[explore_turn['chat_id'] == 165]

,Unnamed: 0,utterance_id,turn_id,chat_id,datetime,user_utterance,valence,label,sublabel,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,valence_new,label_new,sublabel_new,user_utterance_english,bot_utterance_english,Contained technical issue
1826,1826,1864,1,165.0,2023-08-14 16:22:13.302714,start,Follow/Neutral,NaN,NaN,NaN,2023-08-14 16:22:15.507098,"Hallo! Ich bin MIcha, dein Motivations-Chatbot...",NaN,NaN,Follow/Neutral,NaN,NaN,Start,"Hi! I'm MIcha, your motivational chatbot. My g...",NaN
1828,1828,1866,2,165.0,2023-08-14 16:22:47.600064,Ich möchte mich gesünder ernähren,change,Reason,desire,NaN,2023-08-14 16:22:49.288937,Du möchtest also gesünder essen. Kannst du mir...,NaN,NaN,change,Reason,desire,I want to eat healthier,So you want to eat healthier. Can you tell me ...,NaN
1831,1831,1869,3,165.0,2023-08-14 16:25:52.650398,Weil eine gesunde Ernährung positive Auswirkun...,change,Reason,General Reason,NaN,2023-08-14 16:25:55.960667,Wie wichtig ist dir diese Veränderung auf eine...,NaN,NaN,change,Reason,General Reason,Because a healthy diet has a positive impact o...,How important is this change on a scale from 0...,NaN
1832,1832,1870,4,165.0,2023-08-14 16:26:05.341959,7,Follow/Neutral,NaN,NaN,NaN,2023-08-14 16:26:07.17384,Eine 7. Du stehst dieser Veränderung also insg...,NaN,NaN,Follow/Neutral,NaN,NaN,7,A 7. So you're positive about this change over...,NaN
1837,1837,1875,5,165.0,2023-08-14 16:30:13.874548,"Ich habe die obere Einschätzung gewählt, weil ...",Follow/Neutral,NaN,NaN,Querying Extremes,2023-08-14 16:30:19.21889,Interessant. Was sind deiner Meinung nach die ...,gut/hilfreich,NaN,Follow/Neutral,NaN,NaN,I have chosen the upper assessment because I w...,Interesting. What do you think are the best co...,NaN


In [81]:
explore_turn[explore_turn['chat_id'] == 166]

,Unnamed: 0,utterance_id,turn_id,chat_id,datetime,user_utterance,valence,label,sublabel,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,valence_new,label_new,sublabel_new,user_utterance_english,bot_utterance_english,Contained technical issue
1839,1839,1877,1,166.0,2023-08-14 16:31:39.334694,start,Follow/Neutral,NaN,NaN,NaN,2023-08-14 16:31:41.004278,"Hallo! Ich bin MIcha, dein Motivations-Chatbot...",NaN,NaN,Follow/Neutral,NaN,NaN,Start,"Hi! I'm MIcha, your motivational chatbot. My g...",NaN
1840,1840,1878,2,166.0,2023-08-14 16:32:07.853174,Ich möchte mich gesünder ernähren,change,Reason,desire,NaN,2023-08-14 16:32:09.97876,Du möchtest also gesünder essen. Kannst du mir...,NaN,NaN,change,Reason,desire,I want to eat healthier,So you want to eat healthier. Can you tell me ...,NaN
1850,1850,1888,3,166.0,2023-08-14 16:39:10.611345,Diese Veränderung ist gut für meine Gesundheit...,change,Reason,General Reason,NaN,2023-08-14 16:39:13.92619,Wie wichtig ist dir diese Veränderung auf eine...,NaN,NaN,change,Reason,General Reason,This change is good for my health. I will feel...,How important is this change on a scale from 0...,NaN
1851,1851,1889,4,166.0,2023-08-14 16:39:19.478459,6,Follow/Neutral,NaN,NaN,NaN,2023-08-14 16:39:21.196734,Eine 6. Du stehst dieser Veränderung also insg...,NaN,NaN,Follow/Neutral,NaN,NaN,6,A 6. So you're positive about this change over...,NaN
1852,1852,1890,5,166.0,2023-08-14 16:39:48.500971,"Ich habe die obere Einschätzung gewählt, weil ...",Follow/Neutral,NaN,NaN,Elaboration,2023-08-14 16:39:55.415456,Das ist eine sehr nachvollziehbare Perspektive...,gut/hilfreich,NaN,Follow/Neutral,NaN,NaN,I have chosen the upper assessment because I w...,This is a very understandable perspective. Can...,NaN
1856,1856,1894,6,166.0,2023-08-14 16:51:08.02644,"Ich mag Pizza sehr gerne, deshalb möchte ich n...",change,Taking Steps,NaN,Reviewing Past Successes,2023-08-14 16:51:15.966137,"Das ist ein sehr guter Ansatz, um einen Ausgle...",gut/hilfreich,NaN,change,Taking Steps,NaN,"I like pizza very much, so I don't want to do ...",This is a very good approach to finding a bala...,NaN
1857,1857,1895,7,166.0,2023-08-14 16:54:57.90017,nichts besonderes,Follow/Neutral,NaN,NaN,Digging Deep,2023-08-14 16:55:04.667137,Interessant. Kannst du etwas genauer beschreib...,schlecht/unpassend,NaN,Follow/Neutral,NaN,NaN,nothing special,Interesting. Can you describe in more detail w...,NaN
1858,1858,1896,8,166.0,2023-08-14 16:58:20.988251,"Ich mag die Gemüsepizza, aber sie ist nicht gu...",sustain,Reason,desire,Amplified,2023-08-14 16:58:28.196104,"Du scheinst also zu spüren, dass die Gemüsepiz...",gut/hilfreich,NaN,sustain,Reason,desire,"I like the pizza, but it's not good enough to ...",So you seem to feel that while the vegetable p...,NaN
1860,1860,1898,9,166.0,2023-08-14 17:01:43.518876,"Ja, wie kann ich dann dieses Problem lösen?",Follow/Neutral,NaN,NaN,Strength-based,2023-08-14 17:01:51.054863,"Es scheint, als ob du dir bereits Gedanken dar...",schlecht/unpassend,Die Antwort ist überhaupt nicht hilfreich,Follow/Neutral,NaN,NaN,"Yes, then how can I solve this problem?",It seems as if you have already thought about ...,NaN
1862,1862,1900,10,166.0,2023-08-14 17:08:27.112132,"Ich versuche, meine Ernährung ausgewogen zu ge...",change,Commitment,NaN,Summary,2023-08-14 17:08:38.926679,Lass mich unser bisheriges Gespräch zusammenfa...,gut/hilfreich,Gute Zusammenfassung,change,Commitment,NaN,I'm trying to balance my diet. If I eat unheal...,Let me sum up our conversation so far. So you'...,NaN


In [82]:
explore_turn[explore_turn['chat_id'] == 184]

,Unnamed: 0,utterance_id,turn_id,chat_id,datetime,user_utterance,valence,label,sublabel,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,valence_new,label_new,sublabel_new,user_utterance_english,bot_utterance_english,Contained technical issue
2050,2050,2088,1,184.0,2023-08-16 14:42:43.380453,start,Follow/Neutral,NaN,NaN,NaN,2023-08-16 14:42:45.124156,"Hallo! Ich bin MIcha, dein Motivations-Chatbot...",NaN,NaN,Follow/Neutral,NaN,NaN,Start,"Hi! I'm MIcha, your motivational chatbot. My g...",NaN
2052,2052,2090,2,184.0,2023-08-16 14:43:15.637219,Ich möchte nachhaltiger leben,change,Reason,desire,NaN,2023-08-16 14:43:17.24093,Du möchtest also nachhaltiger leben. Kannst du...,NaN,NaN,change,Reason,desire,I want to live more sustainably,So you want to live more sustainably. Can you ...,NaN


In [83]:
explore_turn[explore_turn['chat_id'] == 185]

,Unnamed: 0,utterance_id,turn_id,chat_id,datetime,user_utterance,valence,label,sublabel,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,valence_new,label_new,sublabel_new,user_utterance_english,bot_utterance_english,Contained technical issue
2058,2058,2096,1,185.0,2023-08-16 14:45:18.555109,start,Follow/Neutral,NaN,NaN,NaN,2023-08-16 14:45:20.146678,"Hallo! Ich bin MIcha, dein Motivations-Chatbot...",NaN,NaN,Follow/Neutral,NaN,NaN,Start,"Hi! I'm MIcha, your motivational chatbot. My g...",0.0
2060,2060,2098,2,185.0,2023-08-16 14:45:55.276027,Ich möchte mich gesünder ernähren,change,Reason,desire,NaN,2023-08-16 14:45:57.241207,Du möchtest also gesünder essen. Kannst du mir...,NaN,NaN,change,Reason,desire,I want to eat healthier,So you want to eat healthier. Can you tell me ...,0.0
2062,2062,2100,3,185.0,2023-08-16 14:49:20.452835,"Diese Veränderung ist wichtig, da ich ein gesu...",change,Reason,desire,NaN,2023-08-16 14:49:23.610087,Wie zuversichtlich bist du auf einer Skala von...,NaN,NaN,change,Reason,desire,This change is important because I want to ach...,How confident are you on a scale from 0 (not a...,0.0
2063,2063,2101,4,185.0,2023-08-16 14:49:31.816616,6,Follow/Neutral,NaN,NaN,NaN,2023-08-16 14:49:33.391763,Eine 6. Du stehst dieser Veränderung also insg...,NaN,NaN,Follow/Neutral,NaN,NaN,6,A 6. So you're positive about this change over...,0.0
2064,2064,2102,5,185.0,2023-08-16 14:51:44.180353,Ich möchte eine Balance zwischen gesunder Ernä...,change,Reason,desire,Continuing the Paragraph,2023-08-16 14:51:55.272052,"Du strebst also nach einer Balance, bei der du...",gut/hilfreich,Die Antwort ist korrekt,change,Reason,desire,I want to find a balance between a healthy die...,So you're looking for a balance where you can ...,0.0
2065,2065,2103,6,185.0,2023-08-16 14:53:59.672447,"Ja, das ist korrekt. Hast du Empfehlungen für ...",Follow/Neutral,NaN,NaN,Elaboration,2023-08-16 14:54:14.314581,"Es ist gut zu hören, dass du eine ausgewogene ...",schlecht/unpassend,Die Antwort steht nicht im Zusammenhang mit me...,Follow/Neutral,NaN,NaN,"Yes, that's correct. Do you have any recommend...",It is good to hear that you are looking for a ...,0.0
2066,2066,2104,7,185.0,2023-08-16 15:02:04.360574,"Gestern wollte ich Kuchen essen, aber stattdes...",sustain,Taking Steps,NaN,Exploring Goals and Values,2023-08-16 15:02:09.030896,"Das klingt nach einer bewussten Entscheidung, ...",gut/hilfreich,gute Antwort,sustain,Taking Steps,NaN,"Yesterday I wanted to eat cake, but instead I ...",That sounds like a conscious decision you made...,0.0
2067,2067,2105,8,185.0,2023-08-16 15:05:04.338962,Offensichtlich sind Früchte nicht so lecker wi...,change,Reason,General Reason,Looking Back,2023-08-16 15:05:22.852953,"Es ist großartig zu hören, dass du dich gut ge...",gut/hilfreich,NaN,change,Reason,General Reason,"Obviously fruits are not as tasty as cakes, bu...",It's great to hear that you felt good when you...,0.0
2068,2068,2106,9,185.0,2023-08-16 15:09:13.38618,"Ja, ich erinnere mich, dass ich letzte Woche b...",change,Taking Steps,NaN,Affirmation,2023-08-16 15:09:26.057177,"Es ist bemerkenswert, dass du dich dafür entsc...",gut/hilfreich,NaN,change,Taking Steps,NaN,"Yes, I remember last week I decided to cook at...",It is remarkable that you have chosen to cook ...,0.0
2069,2069,2107,10,185.0,2023-08-16 15:10:39.859764,Danke für dein Kompliment,Follow/Neutral,NaN,NaN,Summary,2023-08-16 15:11:00.268218,Lass mich unser bisheriges Gespräch zusammenfa...,gut/hilfreich,NaN,Follow/Neutral,NaN,NaN,Thanks for your compliment,Let me sum up our conversation so far. You hav...,0.0


#### **Check all the chat_id in duplicate submission**

In [84]:
for i in duplicate_submission_ids:
    chat_ids = explore_user_cleaned[explore_user_cleaned['Submission id'] == i]['chat_id'].to_list()
    print(chat_ids)

[165, 166, 184, 185]
[76, 77, 78]
[]
[18, 20]
[]
[29, 30]
[157, 187]
[43, 47]
[33, 34]
[48, 54]
[]
[]
[160, 163]


#### Use Loops to delete all the duplicate submission in **user_preprocessed.csv** and combine all the chat from same submission id in **turn_preprocessed.csv**

In [85]:
explore_turn_cleaned = explore_turn.copy()

for i in duplicate_submission_ids:
    chat_ids = explore_user_cleaned[explore_user_cleaned['Submission id'] == i]['chat_id'].to_list()
    if len(chat_ids) < 2:
        continue  # Skip if there's no duplicate

    original_id = chat_ids[0]
    ids_to_replace = chat_ids[1:]

    # Replace chat_id in explore_turn with original_id
    explore_turn_cleaned.loc[explore_turn_cleaned['chat_id'].isin(ids_to_replace), 'chat_id'] = original_id

    # Remove non-original chat_id rows from explore_user_cleaned
    explore_user_cleaned = explore_user_cleaned[~explore_user_cleaned['chat_id'].isin(ids_to_replace)]



#### Check result

#### **160 participants in explore_user_cleaned**

In [86]:
submission_id_count = explore_user_cleaned['Submission id'].value_counts()
submission_id_count

Submission id
64d23c118b9d14b04645b779    1
64d252f78e65feea3deec857    1
64d1f6c67356bab250759725    1
64d213a431394162bccabc98    1
64d21167286f5411fc64e5a9    1
                           ..
64dcbe573059a70a9ee178a5    1
64d36441d2b0449574ac9ed9    1
64d3468509eecf544c8584e8    1
64dcdf81195e75dbe6baa84f    1
64dc7992fd5c8149d3b5d4f2    1
Name: count, Length: 160, dtype: int64

#### **160 chat_id in explore_user_cleaned**

In [87]:
explore_user_cleaned['chat_id'].value_counts()


chat_id
62     1
66     1
29     1
45     1
36     1
      ..
181    1
100    1
88     1
188    1
168    1
Name: count, Length: 160, dtype: int64

#### **173 unique id in explore_turn_cleaned**

In [88]:
explore_turn_cleaned['chat_id'].value_counts()

chat_id
165.0    31
76.0     27
160.0    24
48.0     24
18.0     21
         ..
119.0    10
68.0      8
134.0     7
180.0     7
155.0     5
Name: count, Length: 173, dtype: int64

#### find the chat_ids that exist in both explore_user_cleaned and explore_turn_cleaned, and then filters each DataFrame to only keep the rows with those shared chat_ids.

In [89]:
user_chat_ids = set(explore_user_cleaned['chat_id'].unique())
turn_chat_ids = set(explore_turn_cleaned['chat_id'].unique())

common_chat_ids = user_chat_ids & turn_chat_ids  # Get the intersection


explore_user_common = explore_user_cleaned[explore_user_cleaned['chat_id'].isin(common_chat_ids)]
explore_turn_common = explore_turn_cleaned[explore_turn_cleaned['chat_id'].isin(common_chat_ids)]

In [90]:
explore_user_common['chat_id'].value_counts()

chat_id
62     1
66     1
29     1
45     1
36     1
      ..
181    1
100    1
88     1
188    1
168    1
Name: count, Length: 160, dtype: int64

In [91]:
explore_turn_common['chat_id'].value_counts()

chat_id
165.0    31
76.0     27
48.0     24
160.0    24
43.0     21
         ..
64.0     12
65.0     12
66.0     12
67.0     12
138.0    10
Name: count, Length: 160, dtype: int64

#### Calculate `Time taken` in `user_preprocessed.csv`

In [92]:
explore_user_common['start_time'] = pd.to_datetime(explore_user_common['start_time'])
explore_user_common['end_time'] = pd.to_datetime(explore_user_common['end_time'])

explore_user_common['Time taken'] = (
    (explore_user_common['end_time'] - explore_user_common['start_time']).dt.total_seconds() / 60
).round(2) 

print(explore_user_common[['start_time', 'end_time', 'Time taken']])


                    start_time                   end_time  Time taken
0   2023-08-08 14:59:49.246355 2023-08-08 15:24:33.913124       24.74
1   2023-08-08 16:37:15.342634 2023-08-08 17:02:59.270676       25.73
2   2023-08-07 16:25:00.744608 2023-08-07 16:41:43.949226       16.72
3   2023-08-09 12:52:59.707858 2023-08-09 13:04:05.192081       11.09
4   2023-08-09 12:54:12.839555 2023-08-09 13:13:46.172634       19.56
..                         ...                        ...         ...
174 2023-08-10 14:04:26.085861 2023-08-10 14:18:51.144507       14.42
175 2023-08-14 15:37:14.332403 2023-08-14 16:09:01.198191       31.78
177 2023-08-10 12:47:09.320405 2023-08-10 13:05:20.021144       18.18
178 2023-08-08 13:23:23.529339 2023-08-08 13:43:31.954597       20.14
179 2023-08-16 09:25:42.165228 2023-08-16 09:41:10.572300       15.47

[160 rows x 3 columns]


In [93]:
# save as csv
explore_user_common.to_csv('explore_user_common.csv', index=False)
explore_turn_common.to_csv('explore_turn_common.csv', index=False)

#### Translate `user_rating` in `turn_preprocessed.csv`

In [94]:
explore_turn_common['user_rating'].unique()

array([nan, 'gut/hilfreich', 'schlecht/unpassend'], dtype=object)

#### Translate`user_rating`

In [95]:
# Create translation mapping dictionary
rating_translation = {
    'gut/hilfreich': 'good/helpful',
    'schlecht/unpassend': 'bad/inappropriate'
}
explore_turn_common = explore_turn_common.copy()

# Then assign the value
explore_turn_common['user_rating_english'] = explore_turn_common['user_rating'].map(rating_translation)

#### Translate `user_rating_explanation`

In [96]:
import deepl
import time

# DeepL API KEY
# DEEPL_API_KEY = "######"

# Initialise DeepL translator
translator = deepl.Translator(DEEPL_API_KEY)

# translator function
def deepl_translate(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return ""
    
    try:
        result = translator.translate_text(text, source_lang="DE", target_lang="EN-US")
        return result.text
    except Exception as e:
        return f"[Error: {e}]"


translated_texts = []
for i, row in explore_turn_common.iterrows():
    translated = deepl_translate(row['user_rating_explanation'])
    translated_texts.append(translated)
    print(f"Translated row {i}: {translated}")
    time.sleep(0.2)  # Limit the speed

# save it to new column'user_rating_explanation_english'
explore_turn_common['user_rating_explanation_english'] = translated_texts

NameError: name 'DEEPL_API_KEY' is not defined

In [ ]:
explore_turn_common.head(6)

In [ ]:
# save as csv
explore_user_common.to_csv('explore_user_common.csv', index=False)
explore_turn_common.to_csv('explore_turn_common.csv', index=False)